In [1]:
!pip install torch transformers datasets
!pip install numpy==1.26.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [1]:
from google.colab import userdata
import os
os.environ['GIT_TOKEN'] = secret_value = userdata.get('GIT_TOKEN')

In [2]:
!git clone https://$GIT_TOKEN@github.com/Abhishek-P/disrpt25-task.git

Cloning into 'disrpt25-task'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 64 (delta 36), reused 32 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (64/64), 39.25 KiB | 648.00 KiB/s, done.
Resolving deltas: 100% (36/36), done.


In [3]:
%cd '/content/disrpt25-task'
!ls
!pip install conllu

/content/disrpt25-task
disrptdata.py  mapping_disrpt25.json  requirements.txt	util.py
LICENSE        README.MD	      run_model.py


In [4]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
import sys
sys.path.append('/content/disrpt25-task')

Mounted at /content/drive/


In [5]:
from transformers import MT5Tokenizer, MT5ForConditionalGeneration
import torch
import torch.nn as nn
import datasets
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from sklearn.metrics import classification_report, accuracy_score, f1_score
import argparse
import numpy as np
import csv

In [6]:
import disrptdata
disrptdata.DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/sample_data"

In [8]:
# === load dataset ===
"""
experiment with two languages: Chinese and English
"""
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/sample_data"

# load and combine datasets
zho = disrptdata.get_dataset("zho.rst.gcdt")
eng = disrptdata.get_dataset("eng.erst.gum")
combined = disrptdata.get_combined_dataset()
print("Train examples:", combined["train"].num_rows)
print("Dev examples:", combined["dev"].num_rows)

train_dataset = combined['train']
dev_dataset = combined['dev']
train_dataset = train_dataset.class_encode_column('label')
dev_dataset   = dev_dataset.class_encode_column('label')

tokenizer = MT5Tokenizer.from_pretrained("google/mt5-base")

# mappings
lang2id = {'eng': 0, 'zho': 1}
fw2id = {'rst': 0, 'erst': 1}
corpus2id = {'gum': 0, 'gcdt': 1}
dir2id = {'1>2': 0, '1<2': 1}


# preprocess data
def preprocess(example):
    text = f"Classify: Arg1: {example['u1']} Arg2: {example['u2']}"
    encoded = tokenizer(text, padding="max_length", truncation=True, max_length=512)
    encoded["label"] = example["label"]

    # add meta info ids
    encoded["language_ids"] = lang2id[example["lang"]]
    encoded["framework_ids"] = fw2id[example["framework"]]
    encoded["corpus_ids"] = corpus2id[example["corpus"]]
    encoded["direction_ids"] = dir2id[example["direction"]]

    return encoded

# will use batch when dataset size scales up
train_tokenized = train_dataset.map(preprocess, batched=False)
dev_tokenized   = dev_dataset.map(preprocess, batched=False)

train_tokenized.set_format('torch', columns=[
    "input_ids", "attention_mask", "label",
    "language_ids", "framework_ids", "corpus_ids", "direction_ids"
])
dev_tokenized.set_format('torch', columns=[
    "input_ids", "attention_mask", "label",
    "language_ids", "framework_ids", "corpus_ids", "direction_ids"
])

Found the following datasets in the data directory:
Train examples: 4527
Dev examples: 4714


Casting to class labels:   0%|          | 0/4527 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/4714 [00:00<?, ? examples/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.


Map:   0%|          | 0/4527 [00:00<?, ? examples/s]

Map:   0%|          | 0/4714 [00:00<?, ? examples/s]

In [11]:
class MT5Classifier(nn.Module):
    def __init__(self, num_labels, num_languages=2, num_frameworks=2, num_corpora=2, lang_emb_dim=None):
        super().__init__()
        self.encoder = MT5ForConditionalGeneration.from_pretrained("google/mt5-small").get_encoder()
        hidden_size = self.encoder.config.d_model

        # embedding layer for language, corpus, direction, framework
        self.language_embedding = nn.Embedding(num_languages, hidden_size)
        self.framework_embedding = nn.Embedding(num_frameworks, hidden_size)
        self.corpus_embedding = nn.Embedding(num_corpora, hidden_size)
        self.direction_embedding = nn.Embedding(2, hidden_size)  # 0 = 1>2, 1 = 1<2

        # classifier head
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(hidden_size, num_labels)
        self.loss_fct = nn.CrossEntropyLoss()


    def forward(self, input_ids, attention_mask,
            language_ids=None, framework_ids=None, corpus_ids=None, direction_ids=None,
            labels=None):

        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.last_hidden_state.mean(dim=1)  # shape: [batch_size, hidden_size]

        # add metadata embeddings
        if language_ids is not None:
          pooled += self.language_embedding(language_ids)

        if framework_ids is not None:
          pooled += self.framework_embedding(framework_ids)

        if corpus_ids is not None:
          pooled += self.corpus_embedding(corpus_ids)

        if direction_ids is not None:
          pooled += self.direction_embedding(direction_ids)

        logits = self.classifier(pooled)
        loss = self.loss_fct(logits, labels) if labels is not None else None
        return {"loss": loss, "logits": logits}

In [12]:
num_labels = train_tokenized.features["label"].num_classes
num_languages = len(lang2id)
num_frameworks = len(fw2id)
num_corpora = len(corpus2id)

model = MT5Classifier(
    num_labels=num_labels,
    num_languages=num_languages,
    num_frameworks=num_frameworks,
    num_corpora=num_corpora,
    lang_emb_dim=None
)

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [13]:
# === training setup ===
use_cuda = True
device = torch.device("cuda" if torch.cuda.is_available() and use_cuda else "cpu")

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/mt5_embedding/mt5_classifier_embedding.results",
    overwrite_output_dir=False,
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    auto_find_batch_size=True,
)


data_collator = DataCollatorWithPadding(tokenizer)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")

    # get label names from the feature
    label_names = train_tokenized.features["label"].names

    report = classification_report(labels, preds, target_names=label_names)
    print("\n=== Classification Report ===")
    print(report)
    return {"accuracy": acc, "f1": f1}


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=dev_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

batch = next(iter(trainer.get_train_dataloader()))
print("Batch input_ids shape:", batch["input_ids"].shape)

/tmp/ipython-input-13-1832976003.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Batch input_ids shape: torch.Size([2, 512])


In [14]:
trainer.train()
trainer.evaluate()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jw2175 (jw2175-georgetown-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...


=== Classification Report ===
               precision    recall  f1-score   support

  alternation       0.00      0.00      0.00        36
  attribution       0.20      0.13      0.16       318
       causal       0.00      0.00      0.00       238
      comment       0.00      0.00      0.00       180
   concession       0.00      0.00      0.00       189
    condition       0.00      0.00      0.00        93
  conjunction       0.44      0.29      0.35       777
     contrast       0.00      0.00      0.00       199
  elaboration       0.26      0.83      0.39       879
  explanation       0.00      0.00      0.00       382
        frame       0.00      0.00      0.00       235
         mode       0.00      0.00      0.00       104
 organization       0.21      0.78      0.33       315
      purpose       0.00      0.00      0.00       161
        query       0.00      0.00      0.00        79
reformulation       0.00      0.00      0.00       156
     temporal       0.00      0.0

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



=== Classification Report ===
               precision    recall  f1-score   support

  alternation       0.00      0.00      0.00        36
  attribution       0.20      0.13      0.16       318
       causal       0.00      0.00      0.00       238
      comment       0.00      0.00      0.00       180
   concession       0.00      0.00      0.00       189
    condition       0.00      0.00      0.00        93
  conjunction       0.44      0.29      0.35       777
     contrast       0.00      0.00      0.00       199
  elaboration       0.26      0.83      0.39       879
  explanation       0.00      0.00      0.00       382
        frame       0.00      0.00      0.00       235
         mode       0.00      0.00      0.00       104
 organization       0.21      0.78      0.33       315
      purpose       0.00      0.00      0.00       161
        query       0.00      0.00      0.00        79
reformulation       0.00      0.00      0.00       156
     temporal       0.00      0.0

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 2.3361873626708984,
 'eval_accuracy': 0.2632583792957149,
 'eval_f1': 0.16388686296075097,
 'eval_runtime': 59.9086,
 'eval_samples_per_second': 78.686,
 'eval_steps_per_second': 39.343,
 'epoch': 2.0}

In [ ]:
# === error analysis ===
# get predictions on dev set
pred_out = trainer.predict(dev_tokenized)
logits = pred_out.predictions
labels = pred_out.label_ids

# convert logits to label IDs
preds = np.argmax(logits, axis=1)

# softmax confidence score
probs = torch.softmax(torch.tensor(logits), dim=1).numpy()

# extract missclassified examples
dev_texts = dev_tokenized
formatted_texts = [
    f"Arg1: {u1} | Arg2: {u2}"
    for u1, u2 in zip(dev_texts["u1"], dev_texts["u2"])
]

mis = [
    (text, label_encoder.classes_[true], label_encoder.classes_[pred], probs[i][pred])
    for i, (text, true, pred) in enumerate(zip(formatted_texts, labels, preds))
    if true != pred
]

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

out_path = '/content/drive/MyDrive/mt5_vanilla/misclassifications_metainfo.csv'

# log the results to a csv file
with open(out_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["text", "true_label", "pred_label", "confidence"])
    writer.writerows(mis)

print(f"✅ Saved {len(mis)} misclassified examples to:\n{out_path}")